## Simple Process

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../../Data/DATA/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [3]:
X = df.drop('sales', axis=1)
y = df['sales']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Create Model
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [7]:
# Evaluation
from sklearn.metrics import mean_squared_error

MAE = mean_squared_error(y_test, y_pred)
MAE

7.886300641976394

In [8]:
# Adjusting the hyperparameter
model_two = Ridge(alpha=1)
model_two.fit(X_train, y_train)
y_pred_two = model_two.predict(X_test)

In [9]:
# Re-evaluation
MAE_two = mean_squared_error(y_test, y_pred_two)
MAE_two

3.80018250798752

-----
-----
## Cross-Validation

### Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [10]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################
from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=42)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=42)

In [11]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

In [12]:
# Create Model
model = Ridge(alpha=1)
model.fit(X_train,y_train)
y_eval_pred = model.predict(X_eval)

In [13]:
mean_squared_error(y_eval,y_eval_pred)

2.5522105291435966

In [14]:
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

5.048154486831441

----
----
## Cross Validation with cross_val_score(sklearn)

In [ ]:
df = pd.read_csv('../../Data/DATA/Advertising.csv')
df.head()

In [15]:
from sklearn.model_selection import cross_val_score

X = df.drop('sales', axis=1)
y = df['sales']

# TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SCALE DATA
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Create model
model = Ridge(alpha=1)

# SCORING OPTIONS: https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model , X_train, y_train, scoring='neg_mean_squared_error', cv=5)
scores

array([-2.7770397 , -1.58940786, -2.4419187 , -2.20085707, -4.85562156])

In [17]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

2.772968978937123

In [18]:
# Evaluate
model.fit(X_train,y_train)
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

3.80018250798752

----
----
----

## Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

1. It allows specifying multiple metrics for evaluation.

2. It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. 
To evaluate the scores on the training set as well you need to be set to True.

In [19]:
df = pd.read_csv('../../Data/DATA/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [20]:
X = df.drop('sales', axis=1)
y = df['sales']

# TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SCALE DATA
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
from sklearn.model_selection import cross_validate

model = Ridge(alpha=1)
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [22]:
scores

{'fit_time': array([0.05081034, 0.00299931, 0.00513792, 0.00413084, 0.00784993]),
 'score_time': array([0.05281401, 0.00252128, 0.00379848, 0.00389385, 0.01512623]),
 'test_neg_mean_absolute_error': array([-1.28098236, -1.03772447, -1.21457905, -1.30997319, -1.28807619]),
 'test_neg_mean_squared_error': array([-2.7770397 , -1.58940786, -2.4419187 , -2.20085707, -4.85562156]),
 'test_max_error': array([-4.15777701, -3.35566354, -4.14180269, -2.81749293, -9.67324331])}

In [23]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.050810,0.052814,-1.280982,-2.777040,-4.157777
1,0.002999,0.002521,-1.037724,-1.589408,-3.355664
2,0.005138,0.003798,-1.214579,-2.441919,-4.141803
3,0.004131,0.003894,-1.309973,-2.200857,-2.817493
4,0.007850,0.015126,-1.288076,-4.855622,-9.673243


In [24]:
pd.DataFrame(scores).mean()

fit_time                        0.014186
score_time                      0.015631
test_neg_mean_absolute_error   -1.226267
test_neg_mean_squared_error    -2.772969
test_max_error                 -4.829196
dtype: float64

In [25]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [26]:
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

3.80018250798752

-----
-----